<a href="https://colab.research.google.com/github/PedroDS4/Projeto_Processador/blob/main/Mini_compilador_Assembly_Processador_SD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Projeto: Processador de Uso Geral**
O quinto projeto da disciplina de sistemas digitais consiste em um processador de uso geral, que permite ser programado uma memória de instruções e uma memória de dados para a execução do programa.

A tabela abaixo mostra as especificações do processador e suas instruções padrão:


| Oper. | Classe   | Opcode | 4 bits | 4 bits      | 4 bits      | Descrição                                      | Carry | ULA |
|-------|----------|--------|--------|-------------|-------------|------------------------------------------------|--------|-----|
| HLT   | Controle | 0000   | -      | -           | -           | PCₖ₊₁ = PCₖ                                    |        |     |
| LDR   | Dados    | 0001   | A      | addr[7..4]  | addr[3..0]  | Reg[A] ← Memᴰ[addr]                            |        |     |
| STR   | Dados    | 0010   | A      | addr[7..4]  | addr[3..0]  | Memᴰ[addr] ← Reg[A]                            |        |     |
| MOV   | Dados    | 0011   | -      | B           | C           | Reg[B] ← Reg[C]                                |        |     |
| ADD   | ULA      | 0100   | A      | B           | C           | Reg[A] ← Reg[B] + Reg[C]                       |   ●    |  ●  |
| SUB   | ULA      | 0101   | A      | B           | C           | Reg[A] ← Reg[B] - Reg[C]                       |   ●    |  ●  |
| AND   | ULA      | 0110   | A      | B           | C           | Reg[A] ← Reg[B] AND Reg[C]                     |   ●    |  ●  |
| OR    | ULA      | 0111   | A      | B           | C           | Reg[A] ← Reg[B] OR Reg[C]                      |   ●    |  ●  |
| NOT   | ULA      | 1000   | A      | -           | C           | Reg[A] ← NOT Reg[C]                            |   ●    |  ●  |
| XOR   | ULA      | 1001   | A      | B           | C           | Reg[A] ← Reg[B] XOR Reg[C]                     |   ●    |  ●  |
| CMP   | ULA      | 1010   | A      | B           | C           | Reg[A] ← CMP(Reg[B], Reg[C])                  |   ●    |  ●  |
| JMP   | Salto    | 1011   | -      | value[7..4] | value[3..0] | PCₖ₊₁ = value                                  |        |     |
| JNC   | Salto    | 1100   | -      | value[7..4] | value[3..0] | PCₖ₊₁ = value, se carry = 0                   |        |     |
| JC    | Salto    | 1101   | -      | value[7..4] | value[3..0] | PCₖ₊₁ = value, se carry = 1                   |        |     |
| JNZ   | Salto    | 1110   | -      | value[7..4] | value[3..0] | PCₖ₊₁ = value, se ULA ≠ 0                     |        |     |
| JZ    | Salto    | 1111   | -      | value[7..4] | value[3..0] | PCₖ₊₁ = value, se ULA = 0   



Nesse projeto será desenvolvido códigos em assembly, por exemplo, para realizar a multiplicação entre dois números inteiros, como também a conversão de binário para inteiro.




##**Multiplicação de dois números inteiros**
A multiplicação de dois números inteiros pode ser realizada facilmente utilizando um laço em python, como segue


```
M = int(input("Digite o valor do primeiro número"))
N = int(input("Digite o valor do segundo número"))


M_times_N = 0

i = 0
while i != N:
  M_times_N = M_times_N + M
  i = i + 1

```


Em assembly a lógica é parecida, porém os laços são feitos utilizando os Jumps condicionais que existem, assim


```
LDR 0 0       ; R0 = M
LDR 1 1       ; R1 = N
LDR 2 6       ; R2 = 0 (resultado)
LDR 3 6       ; R3 = 0 (contador)
LDR 4 4       ; R4 = 1 (constante 1 para incrementar)

loop:
CMP 5 3 1     ; R5 = R3 - R1
JZ fim        ; se contador == multiplicador, fim
ADD 2 2 0     ; R2 += R0
ADD 3 3 4     ; R3 += 1
JMP loop

fim:
STR 2 2       ; salva resultado em Mem[0x02]
HLT
```


Enquanto em linguagem de máquina(binária), o código é dado por


```
0001 0000  0000 0000
0001 0001  0000 0001
0001 0010  0000 0110
0001 0011  0000 0110
0001 0100  0000 0100


1010 0101  0011 0001

1111 0000  0000 1011

0100 0010  0010 0000

0100 0011  0011 0100


1011 0000  0000 0101


0010 0010  0000 0010

0000 0000  0000 0000

```



No código VHDL, A estrutura para essa memória é definida como uma ROM, dada por

```

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;

entity rom256_16b is
    port (
        clk  : in  std_logic;
        addr : in  std_logic_vector(7 downto 0);
        data : out std_logic_vector(15 downto 0)
    );
end entity rom256_16b;

architecture logica of rom256_16b is

    type rom_array is array (0 to 255) of std_logic_vector(15 downto 0);

    constant ROM_CONTENT : rom_array := (
        -- Endereço | Assembly      | Binário (Linguagem de Máquina)
        0          => B"0001000000000000", -- LDR R0, Mem[0] ; R0 = M
        1          => B"0001000100000001", -- LDR R1, Mem[1] ; R1 = N
        2          => B"0001001000000110", -- LDR R2, Mem[6] ; R2 = 0 (resultado)
        3          => B"0001001100000110", -- LDR R3, Mem[6] ; R3 = 0 (contador i)
        4          => B"0001010000000100", -- LDR R4, Mem[4] ; R4 = 1 (constante)
        
        5          => B"1010010100110001", -- loop: CMP R5, R3, R1
        6          => B"1111000000001011", -- JZ fim (salta para endereço 11 se Z=1)
        7          => B"0100001000100000", -- ADD R2, R2, R0 ; resultado += M
        8          => B"0100001100110100", -- ADD R3, R3, R4 ; contador i++
        9          => B"1011000000000101", -- JMP loop (salta para endereço 5)

        10         => B"0010001000000010", -- fim: STR R2, Mem[2] ; Salva resultado
        11         => B"0000000000000000", -- HLT

        others     => B"0000000000000000"
    );

begin
    process(clk)
    begin
        if rising_edge(clk) then
            data <= ROM_CONTENT(to_integer(unsigned(addr)));
        end if;
    end process;
    
end logica;

```


com os dados armazenados em uma ram que permite a escrita:

```
library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;

entity rom256_16b is
    port (
        clk  : in  std_logic;
        addr : in  std_logic_vector(7 downto 0);
        data : out std_logic_vector(15 downto 0)
    );
end entity rom256_16b;

architecture logica of rom256_16b is

    type rom_array is array (0 to 255) of std_logic_vector(15 downto 0);

    constant ROM_CONTENT : rom_array := (
        -- Endereço | Assembly      | Binário (Linguagem de Máquina)
        0          => B"0001000000000000", -- LDR R0, Mem[0] ; R0 = M
        1          => B"0001000100000001", -- LDR R1, Mem[1] ; R1 = N
        2          => B"0001001000000110", -- LDR R2, Mem[6] ; R2 = 0 (resultado)
        3          => B"0001001100000110", -- LDR R3, Mem[6] ; R3 = 0 (contador i)
        4          => B"0001010000000100", -- LDR R4, Mem[4] ; R4 = 1 (constante)
        
        5          => B"1010010100110001", -- loop: CMP R5, R3, R1
        6          => B"1111000000001011", -- JZ fim (salta para endereço 11 se Z=1)
        7          => B"0100001000100000", -- ADD R2, R2, R0 ; resultado += M
        8          => B"0100001100110100", -- ADD R3, R3, R4 ; contador i++
        9          => B"1011000000000101", -- JMP loop (salta para endereço 5)

        10         => B"0010001000000010", -- fim: STR R2, Mem[2] ; Salva resultado
        11         => B"0000000000000000", -- HLT

        others     => B"0000000000000000"
    );

begin
    process(clk)
    begin
        if rising_edge(clk) then
            data <= ROM_CONTENT(to_integer(unsigned(addr)));
        end if;
    end process;
    
end logica;

```

In [15]:
M = int(input("Digite o valor do primeiro número: "))
N = int(input("Digite o valor do segundo número: "))


M_times_N = 0

i = 0
while i != N:
  M_times_N = M_times_N + M
  i = i + 1


print(M_times_N)

Digite o valor do primeiro número: 15
Digite o valor do segundo número: 3
45



##**Conversão binário para decimal ASCII**
A conversão binária para ASCII pode ser feita utilizando subtrações para contar quantas centenas e quantas dezenas, e o resto dessa subtração total são as unidades do número inteiro


```
# Número binário de entrada (0–255)
numero = int(input("Digite um número inteiro entre 0 e 255: "))

# Constantes (como se estivessem em registradores ou memória)
CENTENA = 100
DEZENA = 10
ASCII_ZERO = 48

# Etapa 1: dividir por 100 (centenas)
resto = numero
centenas = 0
while resto >= CENTENA:
    resto -= CENTENA
    centenas += 1

# Etapa 2: dividir o resto por 10 (dezenas)
dezenas = 0
while resto >= DEZENA:
    resto -= DEZENA
    dezenas += 1

# Etapa 3: o que sobrou é unidade
unidades = resto

# Etapa 4: transformar em ASCII
ascii_centena = centenas + ASCII_ZERO
ascii_dezena = dezenas + ASCII_ZERO
ascii_unidade = unidades + ASCII_ZERO

# Resultado final
print(f"ASCII codes: {ascii_centena}, {ascii_dezena}, {ascii_unidade}")
print(f"Caracteres : {chr(ascii_centena)}, {chr(ascii_dezena)}, {chr(ascii_unidade)}")

```


Em assembly a lógica é parecida, porém os laços são feitos utilizando os Jumps condicionais que existem, assim


```
LDR 0 0       ; R0 = M
LDR 1 1       ; R1 = 100( 01100001 )
LDR 2 2       ; R2 = 10 (00001010)
LDR 3 3       ; R3 = 48 (00110000) zero da tabela ASCII
LDR 4 4       ; R4 = 0 (constante 0)
LDR 5 5       ; R5 = 1 (constante de incremento)

MOV 7 4
MOV 8 4

MOV 6 0
loopcentenas:
CMP 10 6 1     ; R6 = R6 - R1
JC fim_loop_centenas   ;
SUB 6 6 1       ;
ADD 7 7 5     ; R7 += 1
JMP loopcentenas


fim_loop_centenas:
ADD 7 7 3
STR 7 6       ; salva resultado em Mem[0x06]


loopdezenas:
CMP 10 6 2     ; R6 = R6 - R2
JC fim_loop_dezenas  
SUB 6 6 2       ;
ADD 8 8 5     ; R8 += 1
JMP loopdezenas


fim_loop_dezenas:
ADD 8 8 3
STR 8 7       ; salva resultado em Mem[0x07]


ADD 6 6 3
STR 6 8 ; Unidade restante, salva resultado em Mem[0x08]
```


Enquanto em linguagem de máquina(binária), o código é dado por


```
0001 0000  0000 0000
0001 0001  0000 0001
0001 0010  0000 0110
0001 0011  0000 0110
0001 0100  0000 0100


0011 0000  0111 0100
0011 0000  1000 0100




```


No código VHDL, A estrutura para essa memória é definida como uma ROM, dada por

```

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;

entity rom256_16b is
    port (
        clk  : in  std_logic;
        addr : in  std_logic_vector(7 downto 0);
        data : out std_logic_vector(15 downto 0)
    );
end entity rom256_16b;

architecture logica of rom256_16b is

    type rom_array is array (0 to 255) of std_logic_vector(15 downto 0);

    -- Programa de Conversão Binário -> ASCII
    constant ROM_CONTENT : rom_array := (
        0  => B"0001000000000000", -- LDR R0, Mem[0]
        1  => B"0001000100000001", -- LDR R1, Mem[1]
        2  => B"0001001000000010", -- LDR R2, Mem[2]
        3  => B"0001001100000011", -- LDR R3, Mem[3]
        4  => B"0001010000000100", -- LDR R4, Mem[4]
        5  => B"0001010100000101", -- LDR R5, Mem[5]
        6  => B"0011011101000000", -- MOV R7, R4
        7  => B"0011100001000000", -- MOV R8, R4
        8  => B"0011011000000000", -- MOV R6, R0
        9  => B"1010000001100001", -- loop_c: CMP R6, R1
        10 => B"1100000000010000", -- JC fim_c (para 19)
        11 => B"0101011001100001", -- SUB R6, R6, R1
        12 => B"0100011101110101", -- ADD R7, R7, R5
        13 => B"1011000000001001", -- JMP loop_c (para 9)
        14 => B"1010000001100010", -- loop_d: CMP R6, R2
        15 => B"1100000000010101", -- JC fim_d (para 21)
        16 => B"0101011001100010", -- SUB R6, R6, R2
        17 => B"0100100010000101", -- ADD R8, R8, R5
        18 => B"1011000000001110", -- JMP loop_d (para 14)
        19 => B"0100011101110011", -- fim_c: ADD R7, R7, R3
        20 => B"0010011100000110", -- STR R7, Mem[6]
        21 => B"0100100010000011", -- fim_d: ADD R8, R8, R3
        22 => B"0010100000000111", -- STR R8, Mem[7]
        23 => B"0100011001100011", -- ADD R6, R6, R3
        24 => B"0010011000001000", -- STR R6, Mem[8]
        25 => B"0000000000000000", -- HLT
        others => B"0000000000000000"
    );

begin
    process(clk)
    begin
        if rising_edge(clk) then
            data <= ROM_CONTENT(to_integer(unsigned(addr)));
        end if;
    end process;
    
end logica;

```


com os dados armazenados em uma ram que permite a escrita:

```
library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;

entity ram256_8b is
    port (
        clk    : in  std_logic;
        w_en   : in  std_logic;
        addr   : in  std_logic_vector(7 downto 0);
        w_data : in  std_logic_vector(7 downto 0);
        r_data : out std_logic_vector(7 downto 0)
    );
end entity ram256_8b;

architecture ckt of ram256_8b is

    type memoria_ram is array (0 to 255) of std_logic_vector (7 downto 0);

    -- RAM inicializada para o programa de conversão ASCII com valores em binário
    signal RAM : memoria_ram := (
        0      => B"01111011", -- Número a ser convertido: 123
        1      => B"01100100", -- Constante 100
        2      => B"00001010", -- Constante 10
        3      => B"00110000", -- Constante 48 (offset ASCII '0')
        4      => B"00000000", -- Constante 0
        5      => B"00000001", -- Constante 1
        others => B"00000000"
    );
    
begin

    process (clk)
    begin
        if rising_edge(clk) then
            if w_en = '1' then
                RAM(to_integer(unsigned(addr))) <= w_data;
            end if;
        end if;
    end process;
    
    r_data <= RAM(to_integer(unsigned(addr)));

end ckt;

```

In [16]:
# Número binário de entrada (0–255)
numero = int(input("Digite um número inteiro entre 0 e 255: "))

# Constantes (como se estivessem em registradores ou memória)
CENTENA = 100
DEZENA = 10
ASCII_ZERO = 48

# Etapa 1: dividir por 100 (centenas)
resto = numero
centenas = 0
while resto >= CENTENA:
    resto -= CENTENA
    centenas += 1

# Etapa 2: dividir o resto por 10 (dezenas)
dezenas = 0
while resto >= DEZENA:
    resto -= DEZENA
    dezenas += 1

# Etapa 3: o que sobrou é unidade
unidades = resto

# Etapa 4: transformar em ASCII
ascii_centena = centenas + ASCII_ZERO
ascii_dezena = dezenas + ASCII_ZERO
ascii_unidade = unidades + ASCII_ZERO

# Resultado final
print(f"ASCII codes: {ascii_centena}, {ascii_dezena}, {ascii_unidade}")
print(f"Caracteres : {chr(ascii_centena)}, {chr(ascii_dezena)}, {chr(ascii_unidade)}")


Digite um número inteiro entre 0 e 255: 250
ASCII codes: 50, 53, 48
Caracteres : 2, 5, 0


##**Soma de dois números**
Para realizar a soma de dois números, primeiro precisamos realizar os respectivos armazenamento no registrador de calculo do processador, e a ula irá somar os dois números e retornar o valor da soma.

O código assembly para isso é mostrado abaixo:

```
LDR 0 0
LDR 1 1

ADD 2 1 0

STR 2 2
```


Em código binário, temos

```
0001 0000 00000000
0001 0001 00000001

0100 0010 0001 0000

0010 0010 00000010
```





##**Compilador Assembly Para o Projeto**


In [23]:
import numpy as np

# Utilitários para conversão
def reg_bin(reg):
    return format(int(reg[1:]), '04b')

def addr_bin(addr):
    return format(int(addr), '08b')  # pode ser decimal


# 1. MONTADOR
def Assembler(instrucao):
    tokens = instrucao.replace(',', '').split()
    op = tokens[0]

    opcode_dict = {
        'HLT': '0000', 'LDR': '0001', 'STR': '0010', 'MOV': '0011',
        'ADD': '0100', 'SUB': '0101', 'AND': '0110', 'OR': '0111',
        'NOT': '1000', 'XOR': '1001', 'CMP': '1010', 'JMP': '1011',
        'JNC': '1100', 'JC': '1101', 'JNZ': '1110', 'JZ': '1111'
    }

    if op == 'HLT':
        return '0000000000000000'

    opcode = opcode_dict.get(op)
    if opcode is None:
        raise ValueError("Instrução inválida: " + op)

    if op in ['ADD', 'SUB', 'AND', 'OR', 'XOR', 'CMP']:
        A = reg_bin(tokens[1])
        B = reg_bin(tokens[2])
        C = reg_bin(tokens[3])
        return opcode + A + B + C

    elif op == 'NOT':
        A = reg_bin(tokens[1])
        C = reg_bin(tokens[2])
        return opcode + A + '0000' + C

    elif op == 'MOV':
        B = reg_bin(tokens[1])
        C = reg_bin(tokens[2])
        return opcode + '0000' + B + C

    elif op in ['LDR', 'STR']:
        A = reg_bin(tokens[1])
        addr = addr_bin(tokens[2])
        return opcode + A + addr

    elif op in ['JMP', 'JZ', 'JNZ', 'JC', 'JNC']:
        addr = addr_bin(tokens[1])
        return opcode + '0000' + addr

    return None


# 2. FSM (Máquina de Estados)
def FSM_loop(mem_instr, mem_dados):
    PC = 0
    estado_atual = 'BUSCAR'
    registradores = np.zeros(16, dtype=int)
    carry_flag = 0
    zero_flag = 0
    ciclo = 0

    while True:
        print(f"\n[Ciclo {ciclo}] Estado: {estado_atual}")
        ciclo += 1

        if estado_atual == 'BUSCAR':
            instrucao_bin = format(mem_instr[PC], '016b')  # <-- CORREÇÃO PRINCIPAL
            print(f"Instrução: {instrucao_bin}")
            PC += 1
            estado_atual = 'DECODIFICAR'

        elif estado_atual == 'DECODIFICAR':
            opcode = instrucao_bin[:4]
            A = int(instrucao_bin[4:8], 2)
            B = int(instrucao_bin[8:12], 2)
            C = int(instrucao_bin[12:], 2)
            addr = int(instrucao_bin[8:], 2)

            # Mapeamento de opcode para nome de estado
            opcode_estado = {
                '0000': 'HLT', '0001': 'LDR', '0010': 'STR', '0011': 'MOV',
                '0100': 'ADD', '0101': 'SUB', '0110': 'AND', '0111': 'OR',
                '1000': 'NOT', '1001': 'XOR', '1010': 'CMP', '1011': 'JMP',
                '1100': 'JNC', '1101': 'JC', '1110': 'JNZ', '1111': 'JZ'
            }

            estado_atual = opcode_estado.get(opcode, 'HLT')

        elif estado_atual == 'HLT':
            print("Execução finalizada.")
            break

        elif estado_atual == 'ADD':
            registradores[A] = registradores[B] + registradores[C]
            carry_flag = int(registradores[A] > 255)
            registradores[A] &= 0xFF
            zero_flag = int(registradores[A] == 0)
            estado_atual = 'BUSCAR'

        elif estado_atual == 'SUB':
            registradores[A] = registradores[B] - registradores[C]
            carry_flag = int(registradores[B] < registradores[C])
            registradores[A] &= 0xFF
            zero_flag = int(registradores[A] == 0)
            estado_atual = 'BUSCAR'

        elif estado_atual == 'MOV':
            registradores[B] = registradores[C]
            estado_atual = 'BUSCAR'

        elif estado_atual == 'LDR':
            registradores[A] = mem_dados[addr]
            estado_atual = 'BUSCAR'

        elif estado_atual == 'STR':
            mem_dados[addr] = registradores[A]
            estado_atual = 'BUSCAR'

        elif estado_atual == 'JMP':
            PC = addr
            estado_atual = 'BUSCAR'

        elif estado_atual == 'JZ':
            if zero_flag:
                PC = addr
            estado_atual = 'BUSCAR'

        elif estado_atual == 'JNZ':
            if not zero_flag:
                PC = addr
            estado_atual = 'BUSCAR'

        elif estado_atual == 'JC':
            if carry_flag:
                PC = addr
            estado_atual = 'BUSCAR'

        elif estado_atual == 'JNC':
            if not carry_flag:
                PC = addr
            estado_atual = 'BUSCAR'

        elif estado_atual == 'AND':
            registradores[A] = registradores[B] & registradores[C]
            zero_flag = int(registradores[A] == 0)
            estado_atual = 'BUSCAR'

        elif estado_atual == 'OR':
            registradores[A] = registradores[B] | registradores[C]
            zero_flag = int(registradores[A] == 0)
            estado_atual = 'BUSCAR'

        elif estado_atual == 'XOR':
            registradores[A] = registradores[B] ^ registradores[C]
            zero_flag = int(registradores[A] == 0)
            estado_atual = 'BUSCAR'

        elif estado_atual == 'NOT':
            registradores[A] = ~registradores[B] & 0xFF
            zero_flag = int(registradores[A] == 0)
            estado_atual = 'BUSCAR'

        elif estado_atual == 'CMP':
            result = registradores[B] - registradores[C]
            zero_flag = int(result == 0)
            carry_flag = int(registradores[B] < registradores[C])
            estado_atual = 'BUSCAR'

        if PC >= 256:
            print("Fim da memória de instrução.")
            break




# 3. Função Compilar
def compilar(programa, Memoria_dados):
    Memoria_instrucoes = np.zeros(256, dtype=int)
    N_instructions = programa.shape[0]

    for i in range(N_instructions):
        instrucao_bin = Assembler(programa[i])       # string binária
        Memoria_instrucoes[i] = int(instrucao_bin, 2)  # convertido para inteiro

    FSM_loop(Memoria_instrucoes, Memoria_dados)

    return Memoria_instrucoes, Memoria_dados


##**Compilação da Soma de dois números**

In [18]:
import numpy as np

programa = np.array([
    "LDR R0, 0",         # R0 ← Mem[0]
    "LDR R1, 1",         # R1 ← Mem[1]
    "ADD R2, R1, R0",    # R2 ← R1 + R0
    "STR R2, 3",         # Mem[3] ← R2
    "HLT"                # Fim
])

memoria_dados = np.zeros(256, dtype=int)
memoria_dados[0] = 5
memoria_dados[1] = 10

Memoria_instrucoes, Memoria_dados = compilar(programa, memoria_dados)

print(Memoria_dados)



[Ciclo 0] Estado: BUSCAR
Instrução: 0001000000000000

[Ciclo 1] Estado: DECODIFICAR

[Ciclo 2] Estado: LDR

[Ciclo 3] Estado: BUSCAR
Instrução: 0001000100000001

[Ciclo 4] Estado: DECODIFICAR

[Ciclo 5] Estado: LDR

[Ciclo 6] Estado: BUSCAR
Instrução: 0100001000010000

[Ciclo 7] Estado: DECODIFICAR

[Ciclo 8] Estado: ADD

[Ciclo 9] Estado: BUSCAR
Instrução: 0010001000000011

[Ciclo 10] Estado: DECODIFICAR

[Ciclo 11] Estado: STR

[Ciclo 12] Estado: BUSCAR
Instrução: 0000000000000000

[Ciclo 13] Estado: DECODIFICAR

[Ciclo 14] Estado: HLT
Execução finalizada.
[ 5 10  0 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

##**Compilação da Multiplicação de dois números**

In [19]:
Memoria_dados = np.zeros(256, dtype=int)


programa_mul = np.array([
    "LDR R0, 0",       # R0 = M
    "LDR R1, 1",       # R1 = N
    "LDR R2, 6",       # R2 = 0 (resultado)
    "LDR R3, 6",       # R3 = 0 (contador)
    "LDR R4, 4",       # R4 = 1 (constante 1 para incrementar)

    "CMP R5, R3, R1",  # linha 5 - loop
    "JZ 10",           # se contador == multiplicador, fim
    "ADD R2, R2, R0",
    "ADD R3, R3, R4",
    "JMP 5",           # volta para linha do loop (CMP)

    "STR R2, 2",       # linha 10 - fim
    "HLT"
])

M = int(input("Digite o valor do primeiro termo da multiplicação: "))
N = int(input("Digite o valor do segundo termo da multiplicação: "))


Memoria_dados[0] = M
Memoria_dados[1] = N
Memoria_dados[3] = 0
Memoria_dados[4] = 1


Memoria_instrucoes, Memoria_dados = compilar(programa_mul, Memoria_dados)


print(Memoria_dados)







Digite o valor do primeiro termo da multiplicação: 3
Digite o valor do segundo termo da multiplicação: 5

[Ciclo 0] Estado: BUSCAR
Instrução: 0001000000000000

[Ciclo 1] Estado: DECODIFICAR

[Ciclo 2] Estado: LDR

[Ciclo 3] Estado: BUSCAR
Instrução: 0001000100000001

[Ciclo 4] Estado: DECODIFICAR

[Ciclo 5] Estado: LDR

[Ciclo 6] Estado: BUSCAR
Instrução: 0001001000000110

[Ciclo 7] Estado: DECODIFICAR

[Ciclo 8] Estado: LDR

[Ciclo 9] Estado: BUSCAR
Instrução: 0001001100000110

[Ciclo 10] Estado: DECODIFICAR

[Ciclo 11] Estado: LDR

[Ciclo 12] Estado: BUSCAR
Instrução: 0001010000000100

[Ciclo 13] Estado: DECODIFICAR

[Ciclo 14] Estado: LDR

[Ciclo 15] Estado: BUSCAR
Instrução: 1010010100110001

[Ciclo 16] Estado: DECODIFICAR

[Ciclo 17] Estado: CMP

[Ciclo 18] Estado: BUSCAR
Instrução: 1111000000001010

[Ciclo 19] Estado: DECODIFICAR

[Ciclo 20] Estado: JZ

[Ciclo 21] Estado: BUSCAR
Instrução: 0100001000100000

[Ciclo 22] Estado: DECODIFICAR

[Ciclo 23] Estado: ADD

[Ciclo 24] Estado

##**Compilação da Conversão ASCII**

In [30]:
programa_ASCII = np.array([
    "LDR R0, 0",       # R0 = M
    "LDR R1, 1",       # R1 = 100 ('a')
    "LDR R2, 2",       # R2 = 10
    "LDR R3, 3",       # R3 = 48 ('0')
    "LDR R4, 4",       # R4 = 0
    "LDR R5, 5",       # R5 = 1
    "MOV R7, R4",      # R7 = 0
    "MOV R8, R4",      # R8 = 0
    "MOV R6, R0",      # R6 = M (cópia de entrada)
    "CMP R10, R6, R1", # linha 9 - loopcentenas
    "JC 14",           # se R6 < R1, fim_loop_centenas
    "SUB R6, R6, R1",
    "ADD R7, R7, R5",
    "JMP 9",           # volta para loopcentenas
    "ADD R7, R7, R3",  # fim_loop_centenas
    "STR R7, 6",       # salva caractere ASCII da centena
    "CMP R10, R6, R2", # linha 16 - loopdezenas
    "JC 21",           # fim_loop_dezenas
    "SUB R6, R6, R2",
    "ADD R8, R8, R5",
    "JMP 16",          # volta para loopdezenas
    "ADD R8, R8, R3",  # fim_loop_dezenas
    "STR R8, 7",       # salva caractere ASCII da dezena
    "ADD R6, R6, R3",  # converte unidade para ASCII
    "STR R6, 8",        # salva caractere ASCII da unidade
    "HLT"
])


memoria_dados = np.zeros(256, dtype=int)
memoria_dados[0] = 153   # M
memoria_dados[1] = 100   # divisor centenas
memoria_dados[2] = 10    # divisor dezenas
memoria_dados[3] = 48    # ASCII de '0'
memoria_dados[4] = 0
memoria_dados[5] = 1


Memoria_instrucoes, Memoria_dados = compilar(programa_ASCII, memoria_dados)


print(Memoria_dados)





[Ciclo 0] Estado: BUSCAR
Instrução: 0001000000000000

[Ciclo 1] Estado: DECODIFICAR

[Ciclo 2] Estado: LDR

[Ciclo 3] Estado: BUSCAR
Instrução: 0001000100000001

[Ciclo 4] Estado: DECODIFICAR

[Ciclo 5] Estado: LDR

[Ciclo 6] Estado: BUSCAR
Instrução: 0001001000000010

[Ciclo 7] Estado: DECODIFICAR

[Ciclo 8] Estado: LDR

[Ciclo 9] Estado: BUSCAR
Instrução: 0001001100000011

[Ciclo 10] Estado: DECODIFICAR

[Ciclo 11] Estado: LDR

[Ciclo 12] Estado: BUSCAR
Instrução: 0001010000000100

[Ciclo 13] Estado: DECODIFICAR

[Ciclo 14] Estado: LDR

[Ciclo 15] Estado: BUSCAR
Instrução: 0001010100000101

[Ciclo 16] Estado: DECODIFICAR

[Ciclo 17] Estado: LDR

[Ciclo 18] Estado: BUSCAR
Instrução: 0011000001110100

[Ciclo 19] Estado: DECODIFICAR

[Ciclo 20] Estado: MOV

[Ciclo 21] Estado: BUSCAR
Instrução: 0011000010000100

[Ciclo 22] Estado: DECODIFICAR

[Ciclo 23] Estado: MOV

[Ciclo 24] Estado: BUSCAR
Instrução: 0011000001100000

[Ciclo 25] Estado: DECODIFICAR

[Ciclo 26] Estado: MOV

[Ciclo 27]